In [12]:
# Load datasets with proper column names
import pandas as pd
side_effects_columns = ["drug_id", "stereo_id", "umls_id", "meddra_type", "umls_code", "side_effect_name"]
frequencies_columns = ["drug_id", "stereo_id", "umls_id", "meddra_type", "umls_code", "side_effect_name", "frequency", "placebo_frequency"]
indications_columns = ["drug_id", "stereo_id", "umls_id", "meddra_type", "umls_code", "indication"]
drug_names_columns = ["drug_id", "drug_name"]
drug_atc_columns = ["drug_id", "atc_code"]

side_effects = pd.read_csv('meddra_all_se.tsv', sep='\t', names=side_effects_columns, header=None, dtype=str)
frequencies = pd.read_csv('meddra_freq.tsv', sep='\t', names=frequencies_columns, header=None, dtype=str)
indications = pd.read_csv('meddra_all_indications.tsv', sep='\t', names=indications_columns, header=None, dtype=str)
drug_names = pd.read_csv('drug_names.tsv', sep='\t', names=drug_names_columns, header=None, dtype=str)
drug_atc = pd.read_csv('drug_atc.tsv', sep='\t', names=drug_atc_columns, header=None, dtype=str)

In [13]:
side_effects = side_effects[side_effects["meddra_type"] == "PT"]
side_effects = side_effects.drop_duplicates()
side_effects = side_effects.merge(drug_names, on="drug_id", how="left")
side_effects = side_effects.merge(drug_atc, on="drug_id", how="left")

In [14]:
indications_set = set(indications["indication"].unique())
side_effects = side_effects[~side_effects["side_effect_name"].isin(indications_set)]
print("Preprocessed Side Effects Data:")
print(side_effects.head())

Preprocessed Side Effects Data:
         drug_id     stereo_id   umls_id meddra_type umls_code  \
3   CID100000085  CID000010917  C0002418          PT  C0002418   
39  CID100000085  CID000010917  C0025222          PT  C0025222   
44  CID100000085  CID000010917  C0030517          PT  C0030517   
59  CID100000085  CID000010917  C0085595          PT  C0234930   
61  CID100000085  CID000010917  C0151735          PT  C0151735   

           side_effect_name  drug_name atc_code  
3                 Amblyopia  carnitine  A16AA01  
39                  Melaena  carnitine  A16AA01  
44     Parathyroid disorder  carnitine  A16AA01  
59      Skin odour abnormal  carnitine  A16AA01  
61  Injection site reaction  carnitine  A16AA01  


In [95]:
side_effects.shape

(36813, 8)

In [97]:
import os
import requests
import pandas as pd
import gzip
import shutil

In [98]:
side_effects.head()

,drug_id,stereo_id,umls_id,meddra_type,umls_code,side_effect_name,drug_name,atc_code
3,CID100000085,CID000010917,C0002418,PT,C0002418,Amblyopia,carnitine,A16AA01
39,CID100000085,CID000010917,C0025222,PT,C0025222,Melaena,carnitine,A16AA01
44,CID100000085,CID000010917,C0030517,PT,C0030517,Parathyroid disorder,carnitine,A16AA01
59,CID100000085,CID000010917,C0085595,PT,C0234930,Skin odour abnormal,carnitine,A16AA01
61,CID100000085,CID000010917,C0151735,PT,C0151735,Injection site reaction,carnitine,A16AA01


In [99]:
side_effects.shape

(36813, 8)

In [100]:
import requests

drug_name = "Aspirin"
url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/{drug_name}/cids/JSON"

response = requests.get(url)
if response.status_code == 200:
    data = response.json()
    print(data)  # Prints the PubChem CID
else:
    print("Error fetching CID")


{'IdentifierList': {'CID': [2244]}}


In [34]:
import requests

def get_pubchem_properties(cid):
    url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/{cid}/property/MolecularWeight,LogP,TPSA,H_BondDonorCount,H_BondAcceptorCount/JSON"
    
    response = requests.get(url)

    print(f"Status Code: {response.status_code}")  # Debugging
    print(f"Raw Response: {response.text}")  # Print API response

    if response.status_code == 200 and response.text.strip():  
        try:
            data = response.json()
            return data["PropertyTable"]["Properties"][0]  # Extract properties
        except KeyError:
            return "No Properties Found"
    return "Error Fetching Properties"

# Test with a known CID
cid = 2244  # Example CID for Aspirin
properties = get_pubchem_properties(cid)
print(f"Properties for CID {cid}: {properties}")


Status Code: 400
Raw Response: {
  "Fault": {
    "Code": "PUGREST.BadRequest",
    "Message": "Invalid property"
  }
}

Properties for CID 2244: Error Fetching Properties


In [ ]:
def get_chemical_properties(smiles):
    """
    Compute molecular properties using RDKit.
    """
    mol = Chem.MolFromSmiles(smiles)
    if mol:
        return {
            "Molecular Weight": Descriptors.MolWt(mol),
            "LogP": Descriptors.MolLogP(mol),
            "Num Rotatable Bonds": Descriptors.NumRotatableBonds(mol),
            "H Bond Donors": Descriptors.NumHDonors(mol),
            "H Bond Acceptors": Descriptors.NumHAcceptors(mol)
        }
    return None


In [18]:
# Example drug names
drug_names = ["Aspirin", "Ibuprofen", "Paracetamol", "Metformin", "Caffeine"]

# Create a DataFrame
df = pd.DataFrame({"drug_name": drug_names})

# Fetch SMILES using ChEMBL
df["smiles"] = df["drug_name"].apply(get_smiles_from_chembl)

# Compute Chemical Properties
df["chemical_properties"] = df["smiles"].apply(lambda x: get_chemical_properties(x) if x else None)

In [20]:
print(df)

     drug_name smiles chemical_properties
0      Aspirin   None                None
1    Ibuprofen   None                None
2  Paracetamol   None                None
3    Metformin   None                None
4     Caffeine   None                None


In [37]:
import requests
from bs4 import BeautifulSoup

In [15]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time

def get_compound_links_chembl(drug_name):
    """
    Uses Selenium to extract all ChEMBL compound links (/explore/compound/CHEMBLxx) from a search page.
    """
    # Set up Chrome WebDriver
    options = Options()
    options.add_argument("--headless=new")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")

    # Initialize WebDriver
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

    # Construct the ChEMBL search URL
    search_url = f"https://www.ebi.ac.uk/chembl/g/#search_results/all/query={drug_name}"
    driver.get(search_url)

    # Wait for JavaScript to load the page
    time.sleep(5)

    # Find all links on the page
    links = driver.find_elements(By.TAG_NAME, "a")

    # Extract only compound links that contain "/explore/compound/CHEMBL"
    compound_links = []
    for link in links:
        href = link.get_attribute("href")
        if href and "/explore/compound/CHEMBL" in href:
            compound_links.append(href)

    # Close the WebDriver
    driver.quit()

    return compound_links

In [16]:
drug_name = "Aspirin"
compound_links = get_compound_links_chembl(drug_name)

# Display extracted links
for idx, link in enumerate(compound_links):
    print(f"{idx + 1}. {link}")

1. https://www.ebi.ac.uk/chembl/explore/compound/CHEMBL25
2. https://www.ebi.ac.uk/chembl/explore/compound/CHEMBL5282669
3. https://www.ebi.ac.uk/chembl/explore/compound/CHEMBL460292
4. https://www.ebi.ac.uk/chembl/explore/compound/CHEMBL506341
5. https://www.ebi.ac.uk/chembl/explore/compound/CHEMBL454433
6. https://www.ebi.ac.uk/chembl/explore/compound/CHEMBL4085355
7. https://www.ebi.ac.uk/chembl/explore/compound/CHEMBL2103782
8. https://www.ebi.ac.uk/chembl/explore/compound/CHEMBL441043
9. https://www.ebi.ac.uk/chembl/explore/compound/CHEMBL458500
10. https://www.ebi.ac.uk/chembl/explore/compound/CHEMBL1535342
11. https://www.ebi.ac.uk/chembl/explore/compound/CHEMBL457234
12. https://www.ebi.ac.uk/chembl/explore/compound/CHEMBL2105097
13. https://www.ebi.ac.uk/chembl/explore/compound/CHEMBL5314595
14. https://www.ebi.ac.uk/chembl/explore/compound/CHEMBL1697753
15. https://www.ebi.ac.uk/chembl/explore/compound/CHEMBL2260549
16. https://www.ebi.ac.uk/chembl/explore/compound/CHEMBL5314

In [ ]:
# import requests
# from bs4 import BeautifulSoup

# def get_atc_info_and_table(atc_code):
#     """
#     Scrapes WHO ATC/DDD Index for the textual drug description and ATC code table.
#     """
#     url = f"https://www.whocc.no/atc_ddd_index/?code={atc_code}"
#     headers = {"User-Agent": "Mozilla/5.0"}

#     response = requests.get(url, headers=headers)

#     if response.status_code == 200:
#         soup = BeautifulSoup(response.text, "html.parser")

#         # Extract main textual information
#         main_text = []
#         for p in soup.find_all("p"):  # Most textual content is inside <p> tags
#             main_text.append(p.text.strip())

#         # Extract ATC Code table
#         atc_table_data = []
#         atc_table = soup.find("table")  # Locate the first table on the page
        
#         if atc_table:
#             rows = atc_table.find_all("tr")  # Get all rows
            
#             for row in rows:
#                 columns = row.find_all("td")  # Get all columns in the row
                
#                 # Ensure enough columns exist before extracting
#                 if len(columns) >= 5:
#                     atc_table_data.append({
#                         "ATC Code": columns[0].text.strip(),
#                         "Drug Name": columns[1].text.strip(),
#                         "DDD": columns[2].text.strip(),
#                         "Unit": columns[3].text.strip(),
#                         "Administration Route": columns[4].text.strip()
#                     })

#         return "\n".join(main_text), atc_table_data
    
#     return f"Error {response.status_code}: Unable to fetch data", None



In [ ]:

# atc_code = "A16AA01"
# text_info, atc_table = get_atc_info_and_table(atc_code)
# print(f"Textual Information:\n{text_info}\n")
# print(f"ATC Table:\n{atc_table}")

Textual Information:
New search    Hide text from Guidelines
This group comprises all products acting on the alimentary tract and metabolism which cannot be classified in the preceding groups. V03 - All other therapeutic products, should also be considered.Nutrients are classified in V06 - General nutrients.
Agents used in various metabolic deficiency states are classified here, when this is considered to be the main indication e.g. levocarnitine.  Tryptophan and oxitriptan are classified in N06A.Metreleptin used for treatment of complications of leptin deficiency in patients with generalised lipodystrophy is classified in this group.Glutamine for treatment of sickle cell disease is classified here.The DDD of levocarnitine is based on treatment of primary carnitine deficiency.
ATC code  Name  DDD  U Adm.R Note
A16AA01 levocarnitine 2 g O 2 g P List of abbreviations

Last updated: 
2024-12-27
List of abbreviations
Last updated: 
2024-12-27

ATC Table:
[]


In [17]:
def get_first_chembl_link(compound_links):
    """
    Returns the first ChEMBL compound link from the extracted list.
    """
    if compound_links:
        first_link = compound_links[0]
        print(f"✅ First ChEMBL Link: {first_link}")
        return first_link
    else:
        return "⚠️ No compound links found."

In [18]:
# Example: Get ChEMBL compound links for "Aspirin" and verify the correct one
drug_name = "Aspirin"
compound_links = get_compound_links_chembl(drug_name)  # Extracted links from previous function
first_link = get_first_chembl_link(compound_links)

print(first_link)

✅ First ChEMBL Link: https://www.ebi.ac.uk/chembl/explore/compound/CHEMBL25
https://www.ebi.ac.uk/chembl/explore/compound/CHEMBL25


In [50]:
print(side_effects.head(30))

         drug_id     stereo_id   umls_id meddra_type umls_code  \
0   CID100000085  CID000010917  C0000729          PT  C0000737   
1   CID100000085  CID000010917  C0000737          PT  C0687713   
2   CID100000085  CID000010917  C0000737          PT  C0000737   
3   CID100000085  CID000010917  C0002418          PT  C0002418   
4   CID100000085  CID000010917  C0002871          PT  C0002871   
5   CID100000085  CID000010917  C0003123          PT  C0232462   
6   CID100000085  CID000010917  C0003467          PT  C0003467   
7   CID100000085  CID000010917  C0003811          PT  C0003811   
8   CID100000085  CID000010917  C0004093          PT  C0004093   
9   CID100000085  CID000010917  C0004238          PT  C0004238   
10  CID100000085  CID000010917  C0004604          PT  C0004604   
11  CID100000085  CID000010917  C0006277          PT  C0006277   
12  CID100000085  CID000010917  C0007222          PT  C0007222   
13  CID100000085  CID000010917  C0008031          PT  C0008031   
14  CID100

In [89]:
#get me unique drug names from this
drug_names = side_effects["drug_name"].unique()
print(drug_names)
#print number of unique drug_names
print(len(drug_names))

['carnitine' 'gamma-aminobutyric' '5-aminolevulinic' ... 'x'
 'colestyramine' 'K779']
1347


In [19]:
def map_drug_names_to_chembl_links(side_effects_df):
    """
    Loops through all drug names in the side_effects DataFrame and retrieves the first ChEMBL compound link for each.
    """
    chembl_links = []

    for drug_name in side_effects_df["drug_name"].unique():
        print(f"🔎 Searching ChEMBL for: {drug_name}")

        compound_links = get_compound_links_chembl(drug_name)
        first_link = get_first_chembl_link(compound_links)

        print(f"✅ First ChEMBL Link for {drug_name}: {first_link}")
        chembl_links.append({"drug_name": drug_name, "chembl_link": first_link})

    return pd.DataFrame(chembl_links)

In [20]:
chembl_links_df = map_drug_names_to_chembl_links(side_effects)
print(chembl_links_df)

🔎 Searching ChEMBL for: carnitine
✅ First ChEMBL Link: https://www.ebi.ac.uk/chembl/explore/compound/CHEMBL172513
✅ First ChEMBL Link for carnitine: https://www.ebi.ac.uk/chembl/explore/compound/CHEMBL172513
🔎 Searching ChEMBL for: 5-aminolevulinic
✅ First ChEMBL Link: https://www.ebi.ac.uk/chembl/explore/compound/CHEMBL1096562
✅ First ChEMBL Link for 5-aminolevulinic: https://www.ebi.ac.uk/chembl/explore/compound/CHEMBL1096562
🔎 Searching ChEMBL for: leucovorin
✅ First ChEMBL Link: https://www.ebi.ac.uk/chembl/explore/compound/CHEMBL1679
✅ First ChEMBL Link for leucovorin: https://www.ebi.ac.uk/chembl/explore/compound/CHEMBL1679
🔎 Searching ChEMBL for: PGE2
✅ First ChEMBL Link: https://www.ebi.ac.uk/chembl/explore/compound/CHEMBL548
✅ First ChEMBL Link for PGE2: https://www.ebi.ac.uk/chembl/explore/compound/CHEMBL548
🔎 Searching ChEMBL for: prostacyclin
✅ First ChEMBL Link: https://www.ebi.ac.uk/chembl/explore/compound/CHEMBL1139
✅ First ChEMBL Link for prostacyclin: https://www.ebi.a

In [21]:
chembl_links_df

,drug_name,chembl_link
0,carnitine,https://www.ebi.ac.uk/chembl/explore/compound/...
1,5-aminolevulinic,https://www.ebi.ac.uk/chembl/explore/compound/...
2,leucovorin,https://www.ebi.ac.uk/chembl/explore/compound/...
3,PGE2,https://www.ebi.ac.uk/chembl/explore/compound/...
4,prostacyclin,https://www.ebi.ac.uk/chembl/explore/compound/...
...,...,...
1247,tigecycline,https://www.ebi.ac.uk/chembl/explore/compound/...
1248,lymecycline,https://www.ebi.ac.uk/chembl/explore/compound/...
1249,x,https://www.ebi.ac.uk/chembl/explore/compound/...
1250,colestyramine,https://www.ebi.ac.uk/chembl/explore/compound/...


In [22]:
chembl_links_df.to_csv("chembl_links.csv", index=False)

In [23]:
chembl_links_df

,drug_name,chembl_link
0,carnitine,https://www.ebi.ac.uk/chembl/explore/compound/...
1,5-aminolevulinic,https://www.ebi.ac.uk/chembl/explore/compound/...
2,leucovorin,https://www.ebi.ac.uk/chembl/explore/compound/...
3,PGE2,https://www.ebi.ac.uk/chembl/explore/compound/...
4,prostacyclin,https://www.ebi.ac.uk/chembl/explore/compound/...
...,...,...
1247,tigecycline,https://www.ebi.ac.uk/chembl/explore/compound/...
1248,lymecycline,https://www.ebi.ac.uk/chembl/explore/compound/...
1249,x,https://www.ebi.ac.uk/chembl/explore/compound/...
1250,colestyramine,https://www.ebi.ac.uk/chembl/explore/compound/...


In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import re

def extract_chemical_properties_selenium(chembl_link):
    """
    Extracts chemical properties from the Calculated Properties section of a ChEMBL compound page using Selenium.
    """
    # Modify link to point directly to the Calculated Properties section
    properties_link = chembl_link + "#CalculatedProperties"

    # Set up Selenium WebDriver in headless mode
    options = Options()
    options.add_argument("--headless")  # Run in headless mode
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

    # Visit the Calculated Properties section
    driver.get(properties_link)
    time.sleep(5)  # Allow time for JavaScript to fully load

    # Dictionary to store extracted chemical properties
    chemical_properties = {
        "ChEMBL Link": chembl_link
    }

    try:
        # Locate the "Calculated Properties" section
        properties_div = driver.find_element(By.ID, "CalculatedProperties")
        
        # Extract the text from the section
        text_content = properties_div.text.strip()

        # Use regex to extract key-value pairs
        matches = re.findall(r"([\w\s#-]+):\s+([\d\w.-]+)", text_content)
        for match in matches:
            key = match[0].strip()
            value = match[1].strip()
            chemical_properties[key] = value

    except Exception as e:
        print(f"⚠️ Error extracting data from {chembl_link}: {e}")

    driver.quit()
    return chemical_properties

In [2]:
chembl_link = "https://www.ebi.ac.uk/chembl/explore/compound/CHEMBL25"
chemical_data = extract_chemical_properties_selenium(chembl_link)
print(chemical_data)

{'ChEMBL Link': 'https://www.ebi.ac.uk/chembl/explore/compound/CHEMBL25', 'Calculated Properties\nMolecular Weight': '180.16', '0423\nAlogP': '1.31', '#Rotatable Bonds': '2', 'Polar Surface Area': '63.60', 'Molecular Species': 'ACID', 'HBA': '3', 'HBD': '1', '#RO5 Violations': '0', '0\nCX Acidic pKa': '3.41', 'CX Basic pKa': '--', 'CX LogP': '1.24', 'CX LogD': '-2.16', 'Aromatic Rings': '1', 'Heavy Atoms': '13', 'QED Weighted': '0.55', 'Np Likeness Score': '0.12'}


In [62]:
#clean up chembl_links_df
missing_compounds_df = chembl_links_df[chembl_links_df["chembl_link"].str.contains("⚠️ No compound links found.", case=False, na=False)]
print(missing_compounds_df)
missing_compounds_df.shape

                          drug_name                  chembl_link
13                         graphene  ⚠️ No compound links found.
38                        potassium  ⚠️ No compound links found.
220                   anthracycline  ⚠️ No compound links found.
236   diethylenetriaminepentaacetic  ⚠️ No compound links found.
239                        epitopic  ⚠️ No compound links found.
282                      etidronate  ⚠️ No compound links found.
295                       flavoxate  ⚠️ No compound links found.
392                       cytokinin  ⚠️ No compound links found.
571                    procarbazine  ⚠️ No compound links found.
579                        propofol  ⚠️ No compound links found.
673                      tranexamic  ⚠️ No compound links found.
680                           trien  ⚠️ No compound links found.
729                         tetraen  ⚠️ No compound links found.
730                  chloroprocaine  ⚠️ No compound links found.
742                      

(38, 2)

In [77]:
chembl_links_df_clean= chembl_links_df[~chembl_links_df["chembl_link"].str.contains("⚠️ No compound links found.", case=False, na=False)]
print(chembl_links_df_clean)

             drug_name                                        chembl_link
0            carnitine  https://www.ebi.ac.uk/chembl/explore/compound/...
1     5-aminolevulinic  https://www.ebi.ac.uk/chembl/explore/compound/...
2           leucovorin  https://www.ebi.ac.uk/chembl/explore/compound/...
3                 PGE2  https://www.ebi.ac.uk/chembl/explore/compound/...
4         prostacyclin  https://www.ebi.ac.uk/chembl/explore/compound/...
...                ...                                                ...
1247       tigecycline  https://www.ebi.ac.uk/chembl/explore/compound/...
1248       lymecycline  https://www.ebi.ac.uk/chembl/explore/compound/...
1249                 x  https://www.ebi.ac.uk/chembl/explore/compound/...
1250     colestyramine  https://www.ebi.ac.uk/chembl/explore/compound/...
1251              K779  https://www.ebi.ac.uk/chembl/explore/compound/...

[1214 rows x 2 columns]


In [8]:
import pandas as pd
import time
import re
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from urllib.parse import urlparse

def extract_all_chemical_properties(df):
    file_path = "molecular_properties.txt"

    # **Step 1: Read existing entries to prevent duplicate writes**
    try:
        with open(file_path, "r") as f:
            existing_lines = {line.strip() for line in f.readlines()}  # Strip newlines for exact matching
    except FileNotFoundError:
        existing_lines = set()  # If file doesn't exist, start with an empty set

    with open(file_path, "a") as f:  # Open file in append mode
        results = []  # Store extracted data

        for index, row in df.iterrows():
            chembl_link = row["chembl_link"]
            entry_key = f"{row['drug_name']} - {chembl_link}"  # Remove '\n' for proper matching

            if entry_key in existing_lines:
                print(f"⚠️ Skipping duplicate entry: {entry_key}")
                continue

            print(f"🔎 Extracting properties for {row['drug_name']}...")

            # Extract chemical properties
            chemical_data = extract_chemical_properties_selenium(chembl_link)

            if chemical_data:
                chemical_data["Drug Name"] = row["drug_name"]  # Add drug name for reference
                results.append(chemical_data)

                # **Write new entry immediately & flush**
                f.write(entry_key + "\n")
                f.write(str(chemical_data) + "\n")
                f.flush()  # Force write to disk immediately
                existing_lines.add(entry_key)  # Track written entries

            time.sleep(1)  # Avoid getting blocked by ChEMBL

        # Convert results into a DataFrame
        properties_df = pd.DataFrame(results)
        return properties_df



In [15]:
#convert to csv
chembl_links_df_clean.to_csv("chembl_links_clean.csv", index=False)

In [3]:
chembl_links_df_clean = pd.read_csv("chembl_links_clean.csv")

In [ ]:
chemical_properties_df = extract_all_chemical_properties(chembl_links_df_clean)

⚠️ Skipping duplicate entry: carnitine - https://www.ebi.ac.uk/chembl/explore/compound/CHEMBL172513
⚠️ Skipping duplicate entry: 5-aminolevulinic - https://www.ebi.ac.uk/chembl/explore/compound/CHEMBL1096562
⚠️ Skipping duplicate entry: leucovorin - https://www.ebi.ac.uk/chembl/explore/compound/CHEMBL1679
⚠️ Skipping duplicate entry: PGE2 - https://www.ebi.ac.uk/chembl/explore/compound/CHEMBL548
⚠️ Skipping duplicate entry: prostacyclin - https://www.ebi.ac.uk/chembl/explore/compound/CHEMBL1139
⚠️ Skipping duplicate entry: prostaglandin - https://www.ebi.ac.uk/chembl/explore/compound/CHEMBL2074642
⚠️ Skipping duplicate entry: acetylcholine - https://www.ebi.ac.uk/chembl/explore/compound/CHEMBL667
⚠️ Skipping duplicate entry: adenosine - https://www.ebi.ac.uk/chembl/explore/compound/CHEMBL477
⚠️ Skipping duplicate entry: PGE1 - https://www.ebi.ac.uk/chembl/explore/compound/CHEMBL495
⚠️ Skipping duplicate entry: arginine - https://www.ebi.ac.uk/chembl/explore/compound/CHEMBL1485
⚠️ Skipp

In [ ]:
len(results)

NameError: name 'results' is not defined